## 使用Dropout
### 输入层使用Dropout

In [4]:
from sklearn import datasets
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


# 导入数据
dataset = datasets.load_iris()

x = dataset.data
Y = dataset.target

# 设定随机种子
seed = 7
np.random.seed(seed)

# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    #定义Dropout
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

model = KerasClassifier(build_fn=create_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, x, Y, cv=kfold)
print('Accuracy: %.2f%% (%.2f)' % (results.mean()*100, results.std()))

Accuracy: 80.67% (0.21)


### 隐藏层使用

In [6]:
from sklearn import datasets
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


# 导入数据
dataset = datasets.load_iris()

x = dataset.data
Y = dataset.target

# 设定随机种子
seed = 7
np.random.seed(seed)

# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init, kernel_constraint=maxnorm(3)))
    model.add(Dropout(rate=0.2))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init, kernel_constraint=maxnorm(3)))
    model.add(Dropout(rate=0.2))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    # 定义Dropout
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

model = KerasClassifier(build_fn=create_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, x, Y, cv=kfold)
print('Accuracy: %.2f%% (%.2f)' % (results.mean()*100, results.std()))

Accuracy: 88.00% (0.15)


## 学习率的调整
### 线性学习率


In [8]:
from sklearn import datasets
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD


# 导入数据
dataset = datasets.load_iris()

x = dataset.data
Y = dataset.target

# 设定随机种子
seed = 7
np.random.seed(seed)

# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    #模型优化
    learningRate = 0.1
    momentum = 0.9
    decay_rate = 0.005
    sgd = SGD(lr=learningRate, momentum=momentum, decay=decay_rate, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

epochs = 10
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=5, verbose=1)
model.fit(x, Y)

Epoch 1/10
150/150 [==============================] - 0s 888us/step - loss: 0.8722 - accuracy: 0.5267
Epoch 2/10
150/150 [==============================] - 0s 122us/step - loss: 0.4973 - accuracy: 0.6800
Epoch 3/10
150/150 [==============================] - 0s 123us/step - loss: 0.4956 - accuracy: 0.6533
Epoch 4/10
150/150 [==============================] - 0s 119us/step - loss: 0.4706 - accuracy: 0.7000
Epoch 5/10
150/150 [==============================] - 0s 138us/step - loss: 0.4758 - accuracy: 0.6533
Epoch 6/10
150/150 [==============================] - 0s 119us/step - loss: 0.4794 - accuracy: 0.6133
Epoch 7/10
150/150 [==============================] - 0s 132us/step - loss: 0.4774 - accuracy: 0.6600
Epoch 8/10
150/150 [==============================] - 0s 136us/step - loss: 0.4992 - accuracy: 0.6467
Epoch 9/10
150/150 [==============================] - 0s 120us/step - loss: 0.4811 - accuracy: 0.6267
Epoch 10/10
150/150 [==============================] - 0s 132us/step - loss: 0.477

### 指数衰减学习率

In [9]:
from sklearn import datasets
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from math import pow, floor


# 导入数据
dataset = datasets.load_iris()

x = dataset.data
Y = dataset.target

# 设定随机种子
seed = 7
np.random.seed(seed)

# 计算学习率
def step_decay(epoch):
    init_lrate = 0.1
    drop = 0.5
    epochs_drop = 10
    lrate = init_lrate * pow(drop, floor(1 + epoch) / epochs_drop)
    return lrate

# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    #模型优化
    learningRate = 0.1
    momentum = 0.9
    decay_rate = 0.0
    sgd = SGD(lr=learningRate, momentum=momentum, decay=decay_rate, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model


lrate = LearningRateScheduler(step_decay)
epochs = 10
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=5, verbose=1, callbacks=[lrate])
model.fit(x, Y)

Epoch 1/10
150/150 [==============================] - 0s 3ms/step - loss: 0.8796 - accuracy: 0.5267
Epoch 2/10
150/150 [==============================] - 0s 135us/step - loss: 0.4977 - accuracy: 0.6533
Epoch 3/10
150/150 [==============================] - 0s 131us/step - loss: 0.4968 - accuracy: 0.6733
Epoch 4/10
150/150 [==============================] - 0s 129us/step - loss: 0.4716 - accuracy: 0.6800
Epoch 5/10
150/150 [==============================] - 0s 129us/step - loss: 0.4772 - accuracy: 0.6600
Epoch 6/10
150/150 [==============================] - 0s 134us/step - loss: 0.4810 - accuracy: 0.6133
Epoch 7/10
150/150 [==============================] - 0s 151us/step - loss: 0.4786 - accuracy: 0.6533
Epoch 8/10
150/150 [==============================] - 0s 124us/step - loss: 0.5050 - accuracy: 0.6533
Epoch 9/10
150/150 [==============================] - 0s 131us/step - loss: 0.4882 - accuracy: 0.6400
Epoch 10/10
150/150 [==============================] - 0s 133us/step - loss: 0.4806 